In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('data/attacks.csv', encoding='cp1252')
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


<h1> DIRTY DIRTY DATA </h1>

In [3]:
df.shape

(25723, 24)

In [4]:
df.columns  
#las columnas 'Sex ' y alguna más tiene 'espacios' en el nombre de la columna (que resulta algo molesto para trabajar )

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
df.columns = [name.strip() for name in df.columns] 
#Me quito todos los espacios para todos los nombres de las columnas que puedan llevar espacios y poder trabajar mejor  
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

<h3>Checking NULLS</h3>

In [6]:
df_null_COLS = df.isnull().sum().sort_values(ascending=False)
df_null_COLS

# Null  -->  'Age'  --> 22252 
# Null  -->  'Sex'  --> 19986

Unnamed: 22               25722
Unnamed: 23               25721
Time                      22775
Species                   22259
Age                       22252
Sex                       19986
Activity                  19965
Location                  19961
Fatal (Y/N)               19960
Area                      19876
Name                      19631
Country                   19471
Injury                    19449
Investigator or Source    19438
Type                      19425
Year                      19423
href formula              19422
pdf                       19421
href                      19421
Case Number.1             19421
Case Number.2             19421
Date                      19421
original order            19414
Case Number               17021
dtype: int64

In [7]:
df_null_porcentage_COLS = df.isnull().sum().apply(lambda x: x/df.shape[0]).sort_values(ascending=False)
df_null_porcentage_COLS
# Null  -->  'Age'  --> 86% de 'celdas' nulas  🙃
# Null  -->  'Sex'  --> 77% de 'celdas' nulas  🙃

Unnamed: 22               0.999961
Unnamed: 23               0.999922
Time                      0.885394
Species                   0.865335
Age                       0.865062
Sex                       0.776970
Activity                  0.776154
Location                  0.775998
Fatal (Y/N)               0.775959
Area                      0.772694
Name                      0.763169
Country                   0.756949
Injury                    0.756094
Investigator or Source    0.755666
Type                      0.755161
Year                      0.755083
href formula              0.755044
pdf                       0.755005
href                      0.755005
Case Number.1             0.755005
Case Number.2             0.755005
Date                      0.755005
original order            0.754733
Case Number               0.661704
dtype: float64

In [8]:
df_null_porcentage_ROWS = df.isnull().sum(axis=1).apply(lambda x: x/df.shape[1]).sort_values(ascending=False)
df_null_porcentage_ROWS #hay filas con un 100% de nulos  💣💣

12861    1.000000
14368    1.000000
14382    1.000000
14381    1.000000
14380    1.000000
           ...   
1839     0.083333
1840     0.083333
1841     0.083333
1843     0.083333
0        0.083333
Length: 25723, dtype: float64

In [9]:
df_null_ROWS = df.isnull().sum(axis=1).sort_values(ascending=False)
df_null_ROWS    #hay filas que tienen todas las filas Nulas

12861    24
14368    24
14382    24
14381    24
14380    24
         ..
1839      2
1840      2
1841      2
1843      2
0         2
Length: 25723, dtype: int64

In [10]:
df.iloc[15]   #así puedo ver los datos de solo una persona

Case Number                                                      2018.05.12
Date                                                            12-May-2018
Year                                                                   2018
Type                                                             Unprovoked
Country                                                        SOUTH AFRICA
Area                                                  Eastern Cape Province
Location                                          Nahoon Beach, East London
Activity                                                            Surfing
Name                                                                   male
Sex                                                                       M
Age                                                                     NaN
Injury                                         Minor injury, marks on board
Fatal (Y/N)                                                               N
Time        

<h1>1. DEMOGRAPHICS </h1>

<h2>1.1 COL "SEX" </h2>

In [11]:
df['Sex'].unique()   #Podemos ver que no solo tenemos 'F', 'M' o 'NaN' 

array(['F', 'M', nan, 'M ', 'lli', 'N', '.'], dtype=object)

In [12]:
df["Sex"].value_counts(dropna=False)

NaN    19986
M       5094
F        637
M          2
N          2
.          1
lli        1
Name: Sex, dtype: int64

In [13]:
df[(df.Sex =='M ')]  # Aquí quiero ver donde se encuentran el resto de valores sueltos, y ver si les puedo reasignar 

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
563,2014.04.22,22-Apr-2014,2014.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Swimming,male,M,...,NaN,"R. Neale, Florida Today, 4/22/2014",2014.04.22-CocoaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.04.22,2014.04.22,5740.0,NaN,NaN
1587,2005.05.15,15-May-2005,2005.0,Unprovoked,AUSTRALIA,Queensland,50 km east of Townsville,Spearfishing,Ben Edelstein,M,...,Blacktip shark,"J. Anderson, Townsville Bulletin, 5/21/2005",2005.05.15-Edelstein.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2005.05.15,2005.05.15,4716.0,NaN,NaN


In [14]:
df[(df.Sex =='M ')] = 'M'  #Reasignamos 

In [15]:
df["Sex"].value_counts(dropna=False) 
#aquí vemos que ha desaparecido 'M '(que tenía un espacio), vamos ha hacer los mismo para el resto 

NaN    19986
M       5096
F        637
N          2
.          1
lli        1
Name: Sex, dtype: int64

In [16]:
#Averiguamos sus index 
df[(df.Sex == 'N')]     
df[(df.Sex == '.')]     
df[(df.Sex == 'lli')] 

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
1624,2004.11.11.b,11-Nov-2004,2004,Unprovoked,USA,California,"Bunkers, Humboldt Bay, Eureka, Humboldt County",Surfing,Brian Kang,lli,...,5.5 m [18'] white shark,"R. Collier, GSAF",2004.11.11.b-Kang.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2004.11.11.b,2004.11.11.b,4679,NaN,NaN


In [17]:
df.Sex[6131] = 'M'      #es un 'N' y le asiganamos a 'M'  por su 'Name'(Stephen Pettigew)
df.Sex[4938] = 'F'      #es un 'N'                   'F'
df.Sex[5437] = np.nan   #es   '.'                   'NaN' por su 'Name'(NaN)
df.Sex[1624] = 'M'      #es   'lli'                 'M'   por su 'Name'(Brian Kang)

In [18]:
df["Sex"].value_counts(dropna=False)  # Me quedo solo con 3 valores 

NaN    19987
M       5098
F        638
Name: Sex, dtype: int64

<h2>1.2 COL  "AGE"</h2>

In [19]:
df['Age'].value_counts(dropna=False)   #vemos que tenemos 159 datos únos de edad 🤯

NaN         22251
17            154
18            150
19            142
20            141
            ...  
? & 19          1
18 to 22        1
21 or 26        1
81              1
16 to 18        1
Name: Age, Length: 159, dtype: int64

In [20]:
df.Age.unique()   

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'M', 'teen', '77', '74', '28 & 26', '5', '86',
       '18 or 20', '12 or 13', '46 & 34', '28, 23 & 30', 'Teens',
       '36 & 26', '8 or 10', '84', '\xa0 ', ' ', '30 or 36', '6½',
       '21 & ?', '75', '33 or 37', 'mid-30s', '23 & 20', ' 30',
       '7      &    31', ' 28', '20?', "60's", '32 & 30', '16 to 18',
       '87', '67', 'Elderly', 'mid-20s', 'Ca. 33', '74 ', '45 ',
       '21 or 26', '20 ', '>50', '18 to 22', 'adult', '9 & 12', '? & 19',
       '9 months', '2

<h3>PICANDO DATA 🔨</h3>

In [21]:
#En algunos vamos a 'REDONDEAR' al año y otros vamos a sacar la 'MEDIA'
df[(df.Age == '18 months')]
df[(df.Age == '2 to 3 months')]
df[(df.Age == '13 or 14')]
df[(df.Age == '10 or 12')]
df[(df.Age == '31 or 33')]
df[(df.Age == '9 or 10')]
df[(df.Age == '18 or 20')]
df[(df.Age == '36 & 23')]
df[(df.Age == '9 months')]
df[(df.Age == '13 or 18')]
df[(df.Age == '21 & ?')]
df[(df.Age == '21, 34,24 & 35')]
df[(df.Age == '7 or 8')]
df[(df.Age == '37, 67, 35, 27,  ? & 27')]
df[(df.Age == '17 & 16')]
df[(df.Age == '30 & 32')]
df[(df.Age == '33 & 26')]
df[(df.Age == '33 or 37')]    
df[(df.Age == '? & 19')]    
df[(df.Age == '25 or 28')]    
df[(df.Age == '18 to 22')]       
df[(df.Age == '30)')]
df[(df.Age == '')]
df[(df.Age == '0 ')]
df[(df.Age == '74 ')]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
3153,1977.03.13.b,13-Mar-1977,1977,Sea Disaster,AUSTRALIA,Queensland,Near Moreton Island in Moreton Bay,"Their 9 m launch was run down by a 25,000-ton ...",Victor Beaver,M,...,NaN,"Sunday Mail (QLD), 10/11/1992, p.109; A. Sharp...",1977.03.13.b-Beaver.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1977.03.13.b,1977.03.13.b,3150,NaN,NaN


In [22]:
#Reasignando valores
df.Age[445] = '3'  #Año
df.Age[4213] = '1'  
df.Age[6265] = '14'
df.Age[5312] = '11'
df.Age[5483] = '32'
df.Age[4818] = '10'
df.Age[1325] = '19'
df.Age[4999] = '29'
df.Age[3546] = '1'
df.Age[4075] = '15'
df.Age[1925] = '21'
df.Age[3998] = '28'
df.Age[4489] = '8'  #'7 or 8'
df.Age[6063] = '8'  #'7 or 8'
df.Age[3986] = '39' 
df.Age[4587] = '17'
df.Age[4009] = '31'
df.Age[4118] = '29'
df.Age[1949] = '35'
df.Age[3517] = '19'
df.Age[3933] = '26'
df.Age[3375] = '20'
df.Age[1825] = '0'
df.Age[3108] = '0'
df.Age[5084] = '0'
df.Age[3802] = '30'
df.Age[1824] = '30'
df.Age[3153] = '74'

**¿Y SI MEJOR USO REGEX? 🧐🤓**

In [23]:
df['Age'] = df['Age'].str.replace(r'Teen|teen|Teens|"young"|young', "16")
df['Age'] = df['Age'].str.replace(r'adult|"middle-age"|(adult)|mid-30s|30or\s36', "30")
df['Age'] = df['Age'].str.replace(r'MAKE LINE GREEN|\xa0|(M)|X|A.M.|F', '0')
df['Age'] = df['Age'].str.replace(r'Elderly', '50')
df['Age'] = df['Age'].str.replace(r'mid-20s|(20\W)', '20')

df['Age'] = df['Age'].str.replace(r'.s$', '')  # Me cargo a todos aquellos años que van acompañados de una 's' Ej: '50s'
df['Age'] = df['Age'].str.replace(r'½', '')
df['Age'] = df['Age'].str.replace(r'^\W', '')

df['Age'] = df['Age'].str.replace(r'\d+\s+(\w)+\s+\d+', '20')  # Uniendo OR ('30 or 36') y TO ('25 to 35')= media de 20
df['Age'] = df['Age'].str.replace(r'\d+\s+(\W)+\s+\d+', '30')  # Uniendo & ('32 & 30') = media de 30
df['Age'] = df['Age'].str.replace(r'Both\s11', '11')
df['Age'] = df['Age'].str.replace(r'Ca.\s33', '11')
df['Age'] = df['Age'].str.replace(r'\s+&\s+14', '14')
df['Age'] = df['Age'].str.replace(r'00', '0')

df['Age'] = df['Age'].str.replace(r'450', '45')
df['Age'] = df['Age'].str.replace(r'740', '45')
df['Age'] = df['Age'].str.replace(r'28,030', '45')
df['Age'] = df['Age'].str.replace(r'28\,\s30', '45')

In [24]:
df.Age.sort_values(ascending = False).unique()       #😎 y

array(['9', '87', '86', '84', '82', '81', '8', '78', '77', '75', '74',
       '73', '72', '71', '70', '7', '69', '68', '67', '66', '65', '64',
       '63', '62', '61', '60', '6', '59', '58', '57', '56', '55', '54',
       '53', '52', '51', '50', '5', '49', '48', '47', '46', '45 ', '45',
       '44', '43', '42', '41', '40', '4', '39', '38', '37', '36', '35',
       '34', '33', '32', '31', '30', '3', '29', '28', '27', '26', '25',
       '24', '23', '22', '21', '20', '2', '19', '18', '17', '16', '15',
       '14', '13', '12', '11', '10', '1', '0', nan], dtype=object)

In [25]:
df['Age'] = df.Age.apply(float)  #😎🎉🎊 ya tenemos una lista guapa y ya no son strings. 

In [26]:
df['Age'].sort_values(ascending = False)

2846     87.0
922      86.0
1798     84.0
130      82.0
4428     81.0
         ... 
25718     NaN
25719     NaN
25720     NaN
25721     NaN
25722     NaN
Name: Age, Length: 25723, dtype: float64

<h1> 2. SPACE-TIME STATISTICS</h1>

<h2>2.1 COL "DATE" ===> "Year" + "Month"</h2>

In [27]:
df.Date.value_counts(dropna=False)    #DATES  tenemos una de "Length: 5434" 

NaN                     19421
1957                       11
1942                        9
1956                        8
1941                        7
                        ...  
24-Feb-1991                 1
Reported 15-Dec-1877        1
Reported 05-May-1917        1
 27-Mar-2010                1
Reported 08-Jul-1908        1
Name: Date, Length: 5433, dtype: int64

In [28]:
df.Date.unique()

array(['25-Jun-2018', '18-Jun-2018', '09-Jun-2018', ..., '1883-1889',
       '1845-1853', nan], dtype=object)

In [29]:
res = "(\d{4})"  #solo nos quedamos con los AÑOS
df['Year'] = df['Date'].str.extract(pat = res)
df['Year']

0        2018
1        2018
2        2018
3        2018
4        2018
         ... 
25718     NaN
25719     NaN
25720     NaN
25721     NaN
25722     NaN
Name: Year, Length: 25723, dtype: object

In [30]:
df.Year.unique()

array(['2018', '2017', '2016', '2015', '2014', nan, '2013', '2012',
       '2011', '2008', '2010', '2009', '2006', '2007', '2005', '2004',
       '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996',
       '1995', '1984', '1994', '1993', '1992', '1991', '1990', '1989',
       '1988', '1987', '1986', '1985', '1983', '1982', '1981', '1980',
       '1979', '1978', '1977', '1976', '1975', '1974', '1973', '1972',
       '1971', '1970', '1969', '1968', '1967', '1966', '1965', '1964',
       '1954', '1963', '1962', '1961', '1960', '1959', '2176', '1958',
       '1957', '1956', '1955', '1953', '1952', '1951', '1950', '1949',
       '1948', '1848', '1947', '1946', '1945', '1944', '1943', '1942',
       '1941', '1940', '1939', '1938', '1937', '1936', '1935', '1934',
       '1933', '1932', '1931', '1930', '1929', '1928', '1927', '1926',
       '1925', '1924', '1923', '1922', '1921', '1920', '1919', '1918',
       '1917', '1916', '1915', '1914', '1913', '1912', '1911', '1910',
       '1

In [31]:
df.Year.value_counts(dropna=False)

NaN     19442
2015      143
2017      137
2016      131
2011      129
        ...  
1834        1
1791        1
1543        1
1815        1
1580        1
Name: Year, Length: 250, dtype: int64

In [32]:
res_mes = '(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Oct|Nov|Dec)'    #parece interesante tener otra columna que solo sea MESES
df['Month'] = df['Date'].str.extract(pat = res_mes)
df['Month']

0        Jun
1        Jun
2        Jun
3        Jun
4        Jun
        ... 
25718    NaN
25719    NaN
25720    NaN
25721    NaN
25722    NaN
Name: Month, Length: 25723, dtype: object

In [33]:
df.Month.unique()

array(['Jun', 'May', 'Apr', 'Mar', 'Feb', 'Jan', 'Dec', 'Nov', 'Oct', nan,
       'Aug', 'Jul'], dtype=object)

In [34]:
df.Month.value_counts(dropna=False)

NaN    20496
Jul      671
Aug      602
Jan      519
Jun      497
Apr      454
Oct      446
Dec      438
Mar      413
Nov      408
May      397
Feb      382
Name: Month, dtype: int64

<h2> 2.2 COL "LOCATION" ===>  "Place"</h2>

In [35]:
df.Location.value_counts(dropna=False)  #LOCATION, vemos si alguno de estos lugares se encuentran en en los paises 

NaN                                                                          19961
New Smyrna Beach, Volusia County                                               163
Daytona Beach, Volusia County                                                   30
Ponce Inlet, Volusia County                                                     20
Melbourne Beach, Brevard County                                                 18
                                                                             ...  
Cobblestones, Margaret River Area                                                1
Cone Bay                                                                         1
Near mouth of River Cumalarang, 5 miles from Port Isabela, Basilan Island        1
Off Domain, Sydney Harbor                                                        1
Southport, Brunswick County                                                      1
Name: Location, Length: 4109, dtype: int64

In [36]:
df.Location.head(50).unique()

array(['Oceanside, San Diego County', 'St. Simon Island, Glynn County',
       'Habush, Oahu', 'Arrawarra Headland', 'La Ticla',
       'Flat Rock, Ballina', 'Piedade Beach, Recife',
       'Lighhouse Point Park, Ponce Inlet, Volusia County',
       'Cocoa Beach, Brevard  County', 'Daytona Beach, Volusia County',
       'Cairns Aquarium', 'Isle of Palms,  Charleston County',
       'Hilton Head Island, Beaufort County', "Off Land's End",
       'Dugong Bay', 'Nahoon Beach, East London',
       'Sharpes Beach, Ballina', 'Sai Noi Beach', 'Manuelita',
       'Gearys Beach', 'Lennox Head', 'Surfers Point, Prevelly',
       'South Point, Gracetown', 'Madoogali',
       'Robberg Beach, Plettenberg Bay',
       'Shipwreck’s Beach, Keoneloa Bay, Kauai', 'Piedade',
       'Lefthanders, Margaret River Area',
       'Cobblestones, Margaret River Area', 'Nirvana Beach',
       'Praia de Sauaçuhy, Maceió', 'Magenta Beach, Noumea', 'Bimini',
       'St. Francis Bay', 'Kukio Beach', "Waterman's Bay",

In [37]:
res_place = '..(\,.\w+$)'   #Me quedo con el NAME de cada 'Location'
df['Place'] = df.Location.str.extract(pat = res_place)


In [38]:
df.Place.value_counts()

, Maui           74
, Durban         45
, Sydney         41
, Recife         36
, Oahu           20
                 ..
, Macau           1
, Strand          1
, Northcliffe     1
, Manila          1
, Naples          1
Name: Place, Length: 153, dtype: int64

In [39]:
df['Place'] = df['Place'].str.replace(r'(\,)', '')

In [40]:
df.Place.value_counts(dropna=False)

NaN          25223
 Maui           74
 Durban         45
 Sydney         41
 Recife         36
             ...  
 Truk            1
 Lifou           1
 Eilat           1
 Freetown        1
 Búzios          1
Name: Place, Length: 154, dtype: int64

In [41]:
res_place = '(^\w+\,)'      #Me quedo con el NAME de cada 'Location'   
df['Place'] = df.Location.str.extract(pat = res_place)


In [42]:
df.Place.value_counts(dropna=False)

NaN              25365
Ahvaz,              12
Nahoon,              9
Jacksonville,        7
Jupiter,             6
                 ...  
Cobblestones,        1
Hamstead,            1
Offshore,            1
Southgates,          1
Velzyland,           1
Name: Place, Length: 262, dtype: int64

<h1>3. FREE PROYECT </h1>

<h2>COL "SPECIES"</h2>

In [43]:
df.Species.value_counts(dropna=False)  #ESPECIES DE TIBURON

NaN                                                            22258
White shark                                                      163
Shark involvement prior to death was not confirmed               105
Invalid                                                          102
Shark involvement not confirmed                                   88
                                                               ...  
17' white shark                                                    1
Bronze whaler shark, 2.4 m [8'] Identified by Clyde Buttery        1
Zambesi shark, 1.2 m [4']                                          1
Juvenile bull shark                                                1
White shark, 1.5 m [5'] k                                          1
Name: Species, Length: 1551, dtype: int64

In [44]:
df.Species.head(270).unique()

array(['White shark', nan, '2 m shark', 'Tiger shark, 3m', 'Tiger shark',
       "Lemon shark, 3'", "Bull shark, 6'", 'Grey reef shark',
       'Invalid incident', 'Tawny nurse shark, 2m',
       'Shark involvement not confirmed', 'Questionable', '3 m shark',
       'White shark, 3.5 m', 'White shark, 2.5 m', "6' shark",
       'Juvenile bull shark', 'Bull shark', "Tiger shark, 12'",
       'Wobbegong shark', '3.5 m shark', '1.8 m shark', 'Blacktip shark',
       'Juvenile white shark,  2.7 to 3.2 m', 'Bull shark, 2 m',
       'Possibly a wobbegong',
       'Injury believed caused by an eel, not a shark',
       'Galapagos shark?', '2m shark', 'Bull shark, 3 m ',
       'Grey reef shark. 2 m', 'small shark', 'Wobbegong shark?',
       'Juvenile nurse shark', "Nurse shark. 5'", 'Tiger shark, female',
       'Some drowned but other may have been killed by blue sharks',
       'White shark, 4.6 m', 'Cookiecutter shark', 'Wobbegong shark, 1 m',
       'White shark, 4.5 m', 'Spinner shark, 

In [45]:
#res_species = '(+Nurse+ shark|Cookiecutter shark|blue sharks|Bull shark|White+ shark|Lemon shark|Grey reef shark|Tiger shark|Invalid incident|Wobbegong+ shark)'
res_species ='(Lemon|Seven-gil|Bull|Wobbegong|White|Spinner|Tiger|Porbeagle|Nurse|Galapagos|Cookiecutter|shark|Invalid|not|No shark|Angel|reef|sevengill)'
df['Species'] = df.Species.str.extract(pat = res_species)

In [46]:
df.Species.unique()

array(['White', nan, 'shark', 'Tiger', 'Lemon', 'Bull', 'reef', 'Invalid',
       'not', 'Wobbegong', 'Galapagos', 'Nurse', 'Cookiecutter',
       'Spinner', 'Porbeagle', 'No shark', 'Seven-gil', 'sevengill',
       'Angel'], dtype=object)

In [47]:
df.Species.value_counts(dropna=False)

NaN             22485
shark            1848
White             439
Tiger             237
not               225
Bull              133
Invalid           103
reef               60
Nurse              49
Wobbegong          49
Lemon              32
No shark           23
Spinner            14
Galapagos           7
Angel               6
Porbeagle           6
sevengill           4
Seven-gil           2
Cookiecutter        1
Name: Species, dtype: int64

<h2>COL "INJURY"</h2>

In [48]:
df.Injury.value_counts(dropna=False)   #LESIONES - ATAQUES DE TIBURON, la col 'Fatal' no es tan sabrosón como está 

NaN                                                    19449
FATAL                                                    802
Survived                                                  97
Foot bitten                                               87
No injury                                                 82
                                                       ...  
Hamstring bitten                                           1
No injury to occupants, shark bit anchor and ladder        1
Minor injury to foot & ankle                               1
"Very severe wounds"                                       1
FATAL or drowning & scavenging                             1
Name: Injury, Length: 3737, dtype: int64

In [49]:
df.Injury.head(25).unique()

array(['No injury to occupant, outrigger canoe and paddle damaged',
       'Minor injury to left thigh',
       'Injury to left lower leg from surfboard skeg',
       'Minor injury to lower leg',
       'Lacerations to leg & hand shark PROVOKED INCIDENT',
       'No injury, board bitten', 'FATAL',
       'Minor injury to foot. PROVOKED INCIDENT', 'Lower left leg bitten',
       'Minor injury to foot',
       'Minor bite to hand by captive shark. PROVOKED INCIDENT',
       'Injuries to lower right leg and foot',
       'Severe bite to right forearm',
       'Injured by teeth of a dead porbeagle shark he was tossing overboard.',
       'Shallow lacerations to finger PROVOKED INCIDENT',
       'Minor injury, marks on board', 'No injury, surfboard damaged',
       'Minor injury to ankle', 'No injury, shark bit scuba gear',
       'No injury, knocked off board by shark', 'No injury',
       'No injury, shark struck his leg', '5-inch cut to hand',
       'No injury, shark bit hole in ski'], 

In [50]:
res_injury = '(^\w+\s+injury)'
df.Injury = df.Injury.str.extract(pat = res_injury)

In [51]:
df['Injury'] = df.Injury.str.replace(r'minor injury', 'Minor injury')
df['Injury'] = df.Injury.str.replace(r'Serious injury', 'Major injury')
df['Injury'] = df.Injury.str.replace(r'Superficial injury', 'Minor injury')
df['Injury'] = df.Injury.str.replace(r'Severe injury', 'Major injury')
df['Injury'] = df.Injury.str.replace(r'Avulsion injury', 'Major injury')

In [52]:
df.Injury.value_counts(dropna=False)

NaN             24866
No injury         733
Minor injury      112
Major injury       12
Name: Injury, dtype: int64

<h2>COL "TIME"</h2>

In [53]:
df.Time.value_counts(dropna=False) #MOMENTO DEL DÍA EN EL QUE SUCEDIO EL ATAQUE

NaN                            22774
Afternoon                        187
11h00                            128
Morning                          121
12h00                            109
                               ...  
19h35                              1
11h57                              1
11h01 -time of ship sinking        1
"After dark"                       1
After midnight                     1
Name: Time, Length: 368, dtype: int64

In [54]:
df.Time.unique()

array(['18h00', '14h00  -15h00', '07h45', nan, 'Late afternoon', '17h00',
       '14h00', 'Morning', '15h00', '08h15', '11h00', '10h30', '10h40',
       '16h50', '07h00', '09h30', 'Afternoon', '21h50', '09h40', '08h00',
       '17h35', '15h30', '07h30', '19h00, Dusk', 'Night', '16h00',
       '15h01', '12h00', '13h45', '23h30', '09h00', '14h30', '18h30',
       '12h30', '16h30', '18h45', '06h00', '10h00', '10h44', '13h19',
       'Midday', '13h30', '10h45', '11h20', '11h45', '19h30', '08h30',
       '15h45', 'Shortly before 12h00', '17h34', '17h10', '11h15',
       '08h50', '17h45', '13h00', '10h20', '13h20', '02h00', '09h50',
       '11h30', '17h30', '9h00', '10h43', 'After noon', '15h15', '15h40',
       '19h05', '1300', '14h30 / 15h30', '22h00', '16h20', '14h34',
       '15h25', '14h55', '17h46', 'Morning ', '15h49', '19h00',
       'Midnight', '09h30 / 10h00', '10h15', '18h15', '04h00', '14h50',
       '13h50', '19h20', '10h25', '10h45-11h15', '16h45', '15h52',
       '06h15', '14h

In [55]:
res_time = '(^\d\d\w)'
df['Time'] = df['Time'].str.extract(pat = res_time)

In [56]:
df['Time'] = df['Time'].str.replace(r'15j','15h')
df['Time'] = df['Time'].str.replace(r'06j','06h')
df['Time'] = df['Time'].str.replace(r'150','15h')
df['Time'] = df['Time'].str.replace(r'050','05h')
df['Time'] = df['Time'].str.replace(r'160','16h')
df['Time'] = df['Time'].str.replace(r'083','08h')
df['Time'] = df['Time'].str.replace(r'130','13h')

In [57]:
df.Time.unique()

array(['18h', '14h', '07h', nan, '17h', '15h', '08h', '11h', '10h', '16h',
       '09h', '21h', '19h', '12h', '13h', '23h', '06h', '02h', '22h',
       '04h', '05h', '03h', '20h', '00h', '01h'], dtype=object)

<h2>COL "TYPE"</h2> 

In [58]:
df['Type'].value_counts(dropna=False)  #DÓNDE SUCEDIERON LOS ATAQUES

NaN             19425
Unprovoked       4593
Provoked          574
Invalid           547
Sea Disaster      239
Boating           203
Boat              137
M                   2
Questionable        2
Boatomg             1
Name: Type, dtype: int64

In [59]:
df.Type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'M', 'Boatomg'], dtype=object)

In [60]:
df['Type'] = df['Type'].str.replace(r'M','Invalid')
df['Type'] = df['Type'].str.replace(r'Boatomg','Boat')
df['Type'] = df['Type'].str.replace(r'Boating','Boat')
df['Type'] = df['Type'].str.replace(r'Questionable','Unprovoked')

In [61]:
df.Type.value_counts(dropna=False)

NaN             19425
Unprovoked       4595
Provoked          574
Invalid           549
Boat              341
Sea Disaster      239
Name: Type, dtype: int64

<h1> RECAP LIMPIEZA👩‍💻 </h1>

In [62]:
df.head(100)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,Month,Place
0,2018.06.25,25-Jun-2018,2018,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303,NaN,NaN,Jun,"Oceanside,"
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302,NaN,NaN,Jun,NaN
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301,NaN,NaN,Jun,"Habush,"
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300,NaN,NaN,Jun,NaN
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299,NaN,NaN,Jun,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2017.09.02.a,02-Sep-2017,2017,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chase Elmore,M,...,2017.09.02.a-Elmore.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.09.02.a,2017.09.02.a,6208,NaN,NaN,NaN,NaN
96,2017.08.29,29-Aug-2017,2017,Unprovoked,AUSTRALIA,Victoria,Cathedral Rock,Surfing,Marcel Brundler,M,...,2017.08.27-Brundler.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.08.29,2017.08.29,6207,NaN,NaN,Aug,NaN
97,2017.08.27,27-Aug-2017,2017,Unprovoked,USA,Florida,Bathtub Beach,Wading,Violet Veatch,F,...,2017.08.27-Veatch.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.08.27,2017.08.27,6206,NaN,NaN,Aug,NaN
98,2017.08.26.b,26-Aug-2017,2017,Invalid,SPAIN,Castellón,Grao de Moncofa,Swimming,female,F,...,2017.08.26.b-Spain.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.08.26.b,2017.08.26.b,6205,NaN,NaN,Aug,NaN


In [63]:
df1 = df.dropna(axis=0, how='all')
df1.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,Month,Place
0,2018.06.25,25-Jun-2018,2018,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303,NaN,NaN,Jun,"Oceanside,"
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302,NaN,NaN,Jun,NaN
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301,NaN,NaN,Jun,"Habush,"
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300,NaN,NaN,Jun,NaN
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299,NaN,NaN,Jun,NaN


In [64]:
df1.shape

(8703, 26)

In [65]:
df1.Date.value_counts()

1957                         11
1942                          9
1956                          8
1950                          7
1941                          7
                             ..
 27-Mar-2010                  1
04-May-1988                   1
Reported      19-Aug-1993     1
02-Oct-1959                   1
02-Oct-2000                   1
Name: Date, Length: 5432, dtype: int64

<h3> ¿Veamos la cantidad de NaN con la que nos hemos quedado ? 🤡🔎</h3>

In [66]:
df1.columns.value_counts().sum()  # TENGO 26 COLUMNAS EN TOTAL (con las que yo me he creado)

26

In [67]:
df1_null_COLS = df1.isnull().sum().sort_values(ascending=False)   #COLUMNAS
df1_null_COLS

Unnamed: 22               8700
Unnamed: 23               8699
Place                     8345
Injury                    7846
Time                      6351
Species                   5465
Age                       5231
Month                     3476
Sex                       2967
Activity                  2945
Location                  2941
Fatal (Y/N)               2940
Area                      2856
Name                      2611
Country                   2451
Year                      2422
Investigator or Source    2418
Type                      2405
href formula              2402
pdf                       2401
href                      2401
Case Number.1             2401
Case Number.2             2401
Date                      2401
original order            2394
Case Number                  1
dtype: int64

In [68]:
df1_null_porcentage_COLS = df1.isnull().sum().apply(lambda x: x/df1.shape[0]).sort_values(ascending=False)
df1_null_porcentage_COLS        #  % COLUMNAS - hay columnas que podriamos descartar porque casi el 99% de NULOs

Unnamed: 22               0.999655
Unnamed: 23               0.999540
Place                     0.958865
Injury                    0.901528
Time                      0.729748
Species                   0.627944
Age                       0.601057
Month                     0.399403
Sex                       0.340917
Activity                  0.338389
Location                  0.337929
Fatal (Y/N)               0.337815
Area                      0.328163
Name                      0.300011
Country                   0.281627
Year                      0.278295
Investigator or Source    0.277835
Type                      0.276341
href formula              0.275997
pdf                       0.275882
href                      0.275882
Case Number.1             0.275882
Case Number.2             0.275882
Date                      0.275882
original order            0.275078
Case Number               0.000115
dtype: float64

In [69]:
df1_null_COLS = df1.isnull().sum(axis=1).sort_values(ascending=False)   #FILAS
df1_null_COLS   #seguimos viendo filas con muchos nulos

25722    25
7111     25
7109     25
7108     25
7107     25
         ..
884       2
555       2
2893      2
1497      2
0         2
Length: 8703, dtype: int64

In [70]:
df1_null_porcentage_ROWS = df1.isnull().sum(axis=1).apply(lambda x: x/df1.shape[1]).sort_values(ascending=False)
df1_null_porcentage_ROWS    # % FILAS

25722    0.961538
7111     0.961538
7109     0.961538
7108     0.961538
7107     0.961538
           ...   
884      0.076923
555      0.076923
2893     0.076923
1497     0.076923
0        0.076923
Length: 8703, dtype: float64

In [71]:
df1_null_porcentage_ROWS_NULLS = df1_null_porcentage_ROWS[df1_null_porcentage_ROWS < .8].index

<h1> Finish Data cleaning (finitooo 🐱‍🏍) ===> DATAFRAME 2  </h1>

In [72]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23', 'Month', 'Place'],
      dtype='object')

In [73]:
df2 = df1[['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Time','Species','Case Number.1', 'Case Number.2', 'original order', 'Month', 'Place']]

In [74]:
df2.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Time,Species,Case Number.1,Case Number.2,original order,Month,Place
0,2018.06.25,25-Jun-2018,2018,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57.0,No injury,18h,White,2018.06.25,2018.06.25,6303,Jun,"Oceanside,"
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11.0,Minor injury,14h,NaN,2018.06.18,2018.06.18,6302,Jun,NaN
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48.0,NaN,07h,NaN,2018.06.09,2018.06.09,6301,Jun,"Habush,"
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury,NaN,shark,2018.06.08,2018.06.08,6300,Jun,NaN
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,NaN,NaN,Tiger,2018.06.04,2018.06.04,6299,Jun,NaN


In [75]:
df2.columns.value_counts().sum()

19

In [77]:
df2.shape

(8703, 19)

<h1> GUARDAMOS NUESTRA DATA  🔐<h1>

In [83]:
df2.to_csv('Sharks.csv', header=True, index=True)